In [1]:
import os

# Directory with training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

# Directory with validation horse pictures
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Directory with validation human pictures
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

In [2]:
train_horse_names = os.listdir(train_horse_dir)
print(f'TRAIN SET HORSES: {train_horse_names[:10]}')

train_human_names = os.listdir(train_human_dir)
print(f'TRAIN SET HUMANS: {train_human_names[:10]}')

validation_horse_hames = os.listdir(validation_horse_dir)
print(f'VAL SET HORSES: {validation_horse_hames[:10]}')

validation_human_names = os.listdir(validation_human_dir)
print(f'VAL SET HUMANS: {validation_human_names[:10]}')

TRAIN SET HORSES: ['horse04-3.png', 'horse35-3.png', 'horse46-6.png', 'horse01-9.png', 'horse11-6.png', 'horse41-6.png', 'horse43-5.png', 'horse37-5.png', 'horse25-0.png', 'horse16-2.png']
TRAIN SET HUMANS: ['human10-13.png', 'human08-12.png', 'human05-05.png', 'human15-26.png', 'human02-22.png', 'human08-30.png', 'human11-30.png', 'human16-01.png', 'human08-24.png', 'human01-25.png']
VAL SET HORSES: ['horse1-224.png', 'horse1-484.png', 'horse4-556.png', 'horse3-484.png', 'horse2-201.png', 'horse4-501.png', 'horse4-389.png', 'horse2-294.png', 'horse5-076.png', 'horse4-541.png']
VAL SET HUMANS: ['valhuman02-02.png', 'valhuman03-17.png', 'valhuman01-12.png', 'valhuman04-24.png', 'valhuman04-20.png', 'valhuman01-23.png', 'valhuman03-02.png', 'valhuman05-09.png', 'valhuman04-14.png', 'valhuman04-16.png']


In [3]:
print(f'total training horse images: {len(os.listdir(train_horse_dir))}')
print(f'total training human images: {len(os.listdir(train_human_dir))}')
print(f'total validation horse images: {len(os.listdir(validation_horse_dir))}')
print(f'total validation human images: {len(os.listdir(validation_human_dir))}')

total training horse images: 500
total training human images: 527
total validation horse images: 128
total validation human images: 128


In [4]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
#     # The fourth convolution (You can uncomment the 4th and 5th conv layers later to see the effect)
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     # The fifth convolution
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

2022-06-15 19:20:02.551834: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-15 19:20:02.551873: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-15 19:20:06.070634: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-15 19:20:06.070698: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-15 19:20:06.070745: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fadilrisdian-X455LAB): /proc/driver/nvidia/version does not exist
2022-06-15 19:20:06.071151: I tensorflow/core/platform/c

In [5]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=128,
        # Since you used binary_crossentropy loss, you need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        './validation-horse-or-human/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        # Since you used binary_crossentropy loss, you need binary labels
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [7]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 28s 3s/step - loss: 1.2748 - accuracy: 0.5328 - val_loss: 0.7571 - val_accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - 26s 3s/step - loss: 0.7241 - accuracy: 0.7241 - val_loss: 2.0349 - val_accuracy: 0.5000
Epoch 3/15
8/8 [==============================] - 24s 3s/step - loss: 0.6284 - accuracy: 0.7531 - val_loss: 1.0552 - val_accuracy: 0.5469
Epoch 4/15
8/8 [==============================] - 22s 3s/step - loss: 0.2778 - accuracy: 0.8587 - val_loss: 1.1375 - val_accuracy: 0.7578
Epoch 5/15
8/8 [==============================] - 22s 3s/step - loss: 0.1960 - accuracy: 0.9210 - val_loss: 1.1535 - val_accuracy: 0.7461
Epoch 6/15
8/8 [==============================] - 21s 3s/step - loss: 0.0504 - accuracy: 0.9833 - val_loss: 2.3097 - val_accuracy: 0.6875
Epoch 7/15
8/8 [==============================] - 21s 3s/step - loss: 0.9182 - accuracy: 0.8521 - val_loss: 1.0644 - val_accuracy: 0.7500
Epoch 8/15
8/8 [==================